In [1]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import marqo
import pprint

In [2]:
df = pd.read_csv('./data/filevine_case_study_justice - train set_source data set.csv')

def find_tags(txt):
    soup = BeautifulSoup(txt, 'html.parser')
    tags =[tag.name for tag in soup.find_all()]
    return tags

def clean_text(txt):
    soup = BeautifulSoup(txt.strip(), 'html.parser')
    clean_text = soup.get_text()
    clean_text = re.sub('[^a-zA-Z0-9\s]', '', clean_text)
    clean_text = clean_text.lower()
    return clean_text

df['facts'].apply(find_tags).to_csv('tags.csv')

# print(df)

df['facts_clean'] = df['facts'].apply(clean_text)

# print(df['facts_clean'])

# df_dictionary = df[['href', 'facts_clean']].to_dict(orient='records')

# print(df_dictionary)

df[['href', 'facts_clean']]

,href,facts_clean
0,https://api.oyez.org/cases/1984/83-1919,on october 10 1980 an oklahoma city police off...
1,https://api.oyez.org/cases/2009/08-1332,employees of the city of ontario california po...
2,https://api.oyez.org/cases/1976/75-1150,a new jersey statute prohibited the importatio...
3,https://api.oyez.org/cases/1977/77-404,a new jersey law prohibited the importation of...
4,https://api.oyez.org/cases/2004/03-1601,rancho palos verdes a city in california gave ...
...,...,...
2692,https://api.oyez.org/cases/2014/13-628,in 2002 manachem zivotofsky was born in jerusa...
2693,https://api.oyez.org/cases/1992/92-94,james zobrest was deaf since birth he attended...
2694,https://api.oyez.org/cases/2015/14-1418,in 2010 congress passed the affordable care ac...
2695,https://api.oyez.org/cases/2006/05-1508,the department of education certified that the...


In [ ]:
df[df['facts_clean'].str.contains("currently unavailable")]

In [ ]:
df[df['facts_clean'].str.contains("currently unknown")]

In [21]:
clean_df = df[['href', 'facts_clean']].to_dict('records')


In [8]:
type(df[['href', 'facts_clean']].rename(columns={"href": "_id"}).('records'))

NoneType

In [ ]:
print(clean_df)

In [ ]:
type(df_dictionary)

In [ ]:
# mq = marqo.Client(url="http://localhost:8882")

# mq.create_index("my-first-index", model="hf/e5-base-v2")

# mq.index("my-first-index").add_documents(
#     [
#         {
#             "Title": "The Travels of Marco Polo",
#             "Description": "A 13th-century travelogue describing Polo's travels",
#         },
#         {
#             "Title": "Extravehicular Mobility Unit (EMU)",
#             "Description": "The EMU is a spacesuit that provides environmental protection, "
#             "mobility, life support, and communications for astronauts",
#             "_id": "article_591",
#         },
#     ],
#     tensor_fields=["Description"],
# )

# results = mq.index("my-first-index").search(
#     q="What is the best outfit to wear on the moon?"
# )

# print=

In [ ]:
mq = marqo.Client(url="http://localhost:8882")

mq.create_index("test", model="hf/multilingual-e5-large")

mq.index("test").add_documents(
    clean_df,
    tensor_fields=["facts_clean"],
    client_batch_size=50,
    auto_refresh=False
)

In [ ]:
# results = mq.index("test").search(
#     q="he city of Renton, Washington, enacted a zoning ordinance that prohibited adult motion picture",
#     filter_string=['facts_clean']
# )
query = "due process"
results = mq.index("test").search(query, search_method='TENSOR')
print(results)


# Prints the nicely formatted dictionary
pprint.pprint(results)

In [ ]:
mq.index("test").get_stats()

In [ ]:
mq.index("my-first-index").get_documents(
    expose_facets=True,
    document_ids=[]
)

In [22]:
mq.index("sentences_test").delete()

{'acknowledged': True}

In [23]:
mq = marqo.Client(url="http://localhost:8882")

settings = {
    "textPreprocessing": {
        "splitLength": 2,
        "splitOverlap": 0,
        "splitMethod": "sentence",
    },
}

mq.create_index("sentences_test", 
                model="hf/multilingual-e5-large",
                settings_dict=settings
)

mq.index("sentences_test").add_documents(
    clean_df,
    tensor_fields=["facts_clean"],
    client_batch_size=50
)

2024-02-17 18:36:58,686 logger:'marqo' INFO     add_documents batch 0: took 11.564s for Marqo to process & index 50 docs. Roundtrip time: 11.589s.
2024-02-17 18:37:11,561 logger:'marqo' INFO     add_documents batch 1: took 12.861s for Marqo to process & index 50 docs. Roundtrip time: 12.874s.
2024-02-17 18:37:23,659 logger:'marqo' INFO     add_documents batch 2: took 12.034s for Marqo to process & index 50 docs. Roundtrip time: 12.095s.
2024-02-17 18:37:34,952 logger:'marqo' INFO     add_documents batch 3: took 11.280s for Marqo to process & index 50 docs. Roundtrip time: 11.292s.
2024-02-17 18:37:46,100 logger:'marqo' INFO     add_documents batch 4: took 11.136s for Marqo to process & index 50 docs. Roundtrip time: 11.147s.
2024-02-17 18:37:57,538 logger:'marqo' INFO     add_documents batch 5: took 11.423s for Marqo to process & index 50 docs. Roundtrip time: 11.437s.
2024-02-17 18:38:08,161 logger:'marqo' INFO     add_documents batch 6: took 10.611s for Marqo to process & index 50 do

[{'errors': False,
  'processingTimeMs': 11563.79891799952,
  'index_name': 'sentences_test',
  'items': [{'status': 200, '_id': 'cfda625a-45f8-4f14-8a1c-7becf9cf14d3'},
   {'status': 200, '_id': 'd9df6721-0840-40d3-b770-068b5e7e21a8'},
   {'status': 200, '_id': '254621b0-a92f-4a74-9c05-3ad41d1d41cf'},
   {'status': 200, '_id': '21270606-e8aa-41a4-aa8c-72661bd0d06e'},
   {'status': 200, '_id': '67c5e73c-ee0a-45f6-bc96-22a9123388fd'},
   {'status': 200, '_id': 'b0548f38-58ad-4bf5-91b2-89b2d8ec672e'},
   {'status': 200, '_id': '7b0b5197-bfb0-4a4c-bfb9-56ac4e83dbd5'},
   {'status': 200, '_id': '3ac13a11-05dd-4af9-8bd1-a2e003a6eee0'},
   {'status': 200, '_id': 'b993e800-29ad-48f4-8482-54d5eeea2628'},
   {'status': 200, '_id': '525d4127-8975-4ce2-a06f-2fc666afe6c5'},
   {'status': 200, '_id': '63241d81-2d22-47e3-a9ea-2e317c51ce22'},
   {'status': 200, '_id': 'f6bee0c9-6f9d-4fef-b190-f31bf2c9993e'},
   {'status': 200, '_id': '2876d95d-f69e-420b-aafe-036709287403'},
   {'status': 200, '_id': 

In [24]:
mq.index('sentences_test').get_stats()

{'numberOfDocuments': 2697,
 'numberOfVectors': 2697,
 'backend': {'memoryUsedPercentage': 1.50920197244,
  'storageUsedPercentage': 9.1102836551}}

In [26]:
# results = mq.index("test").search(
#     q="he city of Renton, Washington, enacted a zoning ordinance that prohibited adult motion picture",
#     filter_string=['facts_clean']
# )
query = "A New Jersey law prohibited the importation of most solid or liquid waste which originated or was collected outside the territorial limits of the State."
results = mq.index("sentences_test").search(query, search_method='TENSOR')

# Prints the nicely formatted dictionary
pprint.pprint(results)

{'hits': [{'_highlights': [{'facts_clean': 'a new jersey law prohibited the '
                                           'importation of most solid or '
                                           'liquid waste which originated or '
                                           'was collected outside the '
                                           'territorial limits of the state'}],
           '_id': '21270606-e8aa-41a4-aa8c-72661bd0d06e',
           '_score': 0.989584693033661,
           'facts_clean': 'a new jersey law prohibited the importation of most '
                          'solid or liquid waste which originated or was '
                          'collected outside the territorial limits of the '
                          'state',
           'href': 'https://api.oyez.org/cases/1977/77-404'},
          {'_highlights': [{'facts_clean': 'a new jersey statute prohibited '
                                           'the importation of solid or liquid '
                             

In [41]:
pd.DataFrame(results['hits']).loc[1]._highlights


[{'facts_clean': 'a new jersey statute prohibited the importation of solid or liquid waste into the state except for garbage for swine feed the city of philadelphia challenged the statute alleging it was unconstitutional under the commerce clause of article i and preempted by the solid waste disposal act of 1965 the new jersey supreme court upheld the statute congress then passed the resource conservation and recovery act of 1976'}]